In [1]:
import os
from copy import deepcopy

from tqdm import tqdm
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

from Environment import Environment, make_one_hot, give_mapping
from PrioritizedReplayBuffer import ReplayBuffer
from Networks import UserActor, AsstActor, CentralizedCritic

In [2]:
def give_prev_steps(prev_steps_assist, steps):
    prev_steps_assist = [[0,0,0,0,-1,-1] for i in range(steps-1)]
    return prev_steps_assist

In [3]:
class Agent:
    def __init__(self):
        self.user_actor = UserActor()
        self.asst_actor = AsstActor()
        self.critic = CentralizedCritic()
        self.optimizer_actors = tf.keras.optimizers.Adam(lr = 0.0001)
        self.optimizer_critic = tf.keras.optimizers.Adam(lr = 0.0002)
        self.huber_loss = tf.keras.losses.Huber()
        self.memory_len = 4
        
        self.gamma = 0.95
        self.env = Environment()
        self.env.cells = np.array([[0.7, 0.1], [0.1, 0.1], [0.5, 0.7], [0.6, 0.2], [0.7, 0.4], [0.2, 0.9]])
        self.env_cell_mapping = give_mapping(self.env.cells)
        self.env_cell_mapping = self.env_cell_mapping[np.newaxis, :, :, np.newaxis]
        self.eps = 10e-6
    
    def learn(self):
        env = self.env
        max_steps = 200
        running_reward = 0
        reached = 0
        
        for epoch in tqdm(range(100000)):
#             input_As = [] #Input of user agent
#             input_Bs = [] #Input of assitant agent/output of user agent
#             input_Cs = [] #Icon layout
#             input_Ds = [] #Output of assitant agent
            
            user_action_probs_history = []
            asst_action_probs_history = []
            critic_value_history = []
            rewards_history = []
            returns = [] #Returns
            
            done = False
            episode_reward = 0
            start, dest = env.give_start_dest()
            ob_user = [start[0], start[1], dest[0], dest[1]]
            prev_steps_assist = []
            prev_steps_assist = give_prev_steps(prev_steps_assist, self.memory_len)
            step = 0
            episode_reward = 0
            
            with tf.GradientTape(persistent = True) as tape:
                while not done and step<max_steps:
                    curr_loc = ob_user[:2]
                    target_loc = ob_user[2:4]
                    step+=1
                    ob_user = np.array(ob_user)[np.newaxis]
                    user_probs = self.user_actor.model(ob_user)
                    user_action = np.random.choice(4, p=np.squeeze(user_probs))
                    user_action_probs_history.append(tf.math.log(user_probs[0, user_action]))

                    action_user_one_hot = make_one_hot(user_action, 4)

                    ob_assist = [action_user_one_hot + curr_loc] 
                    ob_assist = prev_steps_assist + ob_assist
                    ob_assist = np.array(ob_assist)[np.newaxis]
                    
                    asst_probs = self.asst_actor.model([ob_assist, self.env_cell_mapping])
                    asst_action = np.random.choice(4, p=np.squeeze(asst_probs))
                    asst_action_probs_history.append(tf.math.log(asst_probs[0, asst_action]))
                    
                    asst_output_one_hot = np.array(make_one_hot(asst_action, 4))[np.newaxis]
                    
                    critic_value = self.critic.model([ob_user, ob_assist, self.env_cell_mapping, asst_output_one_hot])
                    critic_value_history.append(critic_value)
                    
                    new_loc, reward_user, reward_assist, done = self.env.step(user_action, asst_action + 1, target_loc, curr_loc)
                    
                    next_ob_user = new_loc[:]
                    next_ob_user = next_ob_user + target_loc

                    ob_user = next_ob_user
                    prev_steps_assist = np.squeeze(ob_assist).tolist()[1:]
                    
                    rewards_history.append(reward_user)
                    episode_reward+=reward_user
                    
                    if done:
                        reached += 1

                running_reward = 0.05 * episode_reward + (1 - 0.05) * running_reward

                discounted_sum = 0
                for r in rewards_history[::-1]:
                    discounted_sum = r + self.gamma * discounted_sum
                    returns.append(discounted_sum)
                returns.reverse()
                
                returns = np.array(returns)
                returns = (returns - np.mean(returns)) / (np.std(returns) + self.eps)
                returns = returns.tolist()
                
                critic_losses = []
                user_losses = []
                asst_losses = []
                
                for log_prob_user, log_prob_asst, val, ret in zip(user_action_probs_history, asst_action_probs_history, critic_value_history,\
                                                                 returns):
                    diff = ret - val
                    user_losses.append(-log_prob_user*diff)
                    asst_losses.append(-log_prob_asst*diff)
                    critic_losses.append(self.huber_loss(tf.expand_dims(val, 0), tf.expand_dims(ret, 0)))

                user_loss = sum(user_losses)
                asst_loss = sum(asst_losses)
                critic_loss = sum(critic_losses)

            grads = tape.gradient(user_loss, self.user_actor.model.trainable_variables)
            self.optimizer_actors.apply_gradients(zip(grads, self.user_actor.model.trainable_variables))

            grads = tape.gradient(asst_loss, self.asst_actor.model.trainable_variables)
            self.optimizer_actors.apply_gradients(zip(grads, self.asst_actor.model.trainable_variables))

            grads = tape.gradient(critic_loss, self.critic.model.trainable_variables)
            self.optimizer_critic.apply_gradients(zip(grads, self.critic.model.trainable_variables))
            
            if epoch%100 == 0:
                print(running_reward)
            
            if epoch and epoch%100 == 0:
                print(reached)
                reached = 0
                self.trial()
                
    def trial(self):
        env = self.env
        max_steps = 40
        done = False
        episode_reward = 0
        start, dest = env.give_start_dest()
        ob_user = [start[0], start[1], dest[0], dest[1]]
        prev_steps_assist = []
        prev_steps_assist = give_prev_steps(prev_steps_assist, self.memory_len)
        step = 0
        episode_reward = 0

        while not done and step<max_steps:
            curr_loc = ob_user[:2]
            target_loc = ob_user[2:4]
            step+=1
            print(ob_user)

            ob_user = np.array(ob_user)[np.newaxis]
            user_probs = self.user_actor.model(ob_user)
            user_action = np.argmax(np.squeeze(user_probs))

            action_user_one_hot = make_one_hot(user_action, 4)

            ob_assist = [action_user_one_hot + curr_loc] 
            ob_assist = prev_steps_assist + ob_assist
            ob_assist = np.array(ob_assist)[np.newaxis]

            asst_probs = self.asst_actor.model([ob_assist, self.env_cell_mapping])
            asst_action = np.argmax(np.squeeze(asst_probs))

            asst_output_one_hot = np.array(make_one_hot(asst_action, 4))[np.newaxis]

            critic_value = self.critic.model([ob_user, ob_assist, self.env_cell_mapping, asst_output_one_hot])

            new_loc, reward_user, reward_assist, done = self.env.step(user_action, asst_action + 1, target_loc, curr_loc)

            next_ob_user = new_loc[:]
            next_ob_user = next_ob_user + target_loc

            ob_user = next_ob_user
            prev_steps_assist = np.squeeze(ob_assist).tolist()[1:]
            episode_reward+=reward_user




In [4]:
agent = Agent()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice (Tens [(None, 2)]          0           input_1[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_strided_slice_1 (Te [(None, 2)]          0           input_1[0][0]                    
__________________________________________________________________________________________________
subtract (Subtract)             (None, 2)            0           tf_op_layer_strided_slice[0][0]  
                                                                 tf_op_layer_strided_sl

In [ ]:
agent.learn()

  0%|                                                                           | 1/100000 [00:09<255:09:14,  9.19s/it]

-6.2


  0%|                                                                           | 5/100000 [00:26<158:53:25,  5.72s/it]

In [ ]:
agent.trial()